In [2]:
#import requests
import random
#import time
import numpy as np
import pandas as pd
#from bs4 import BeautifulSoup as bs
import re
from sklearn.preprocessing import LabelEncoder
#from os import path   #uncomment these if you have downloaded and installed wordcloud based on the instructions above
#from PIL import Image
#from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import sparse
from sklearn.metrics import pairwise_distances
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from sklearn.metrics import multilabel_confusion_matrix
#from imblearn.over_sampling import ADASYN
import joblib

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## Importing X_legit and y which contain the shops that userid 2043 rated and ratings respectively
---

In [3]:
X_legit = pd.read_csv('xlegit.csv')
X_legit.shape

(980, 19307)

In [4]:
y = pd.read_csv('y.csv', squeeze=True)
y.head()

0    4.0
1    4.0
2    5.0
3    5.0
4    4.0
Name: user_rating, dtype: float64

In [5]:
#split the dataset into train and test sets first
X_train, X_test, y_train, y_test = train_test_split(X_legit, y, test_size=0.2, random_state=42, stratify=y)

In [6]:
y_train.value_counts(normalize=True)

4.0    0.479592
5.0    0.383929
3.0    0.118622
2.0    0.012755
1.0    0.005102
Name: user_rating, dtype: float64

In [7]:
#instantiate scaler since not all of the features are of the same scale, eg. review_count and avg_store_rating
ss = StandardScaler()

In [8]:
#fitting the train and transforming both the train and test sets
X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

In [10]:
# Instantiate the model
xgb_model = XGBClassifier()

In [11]:
#just in case it takes too long to run this even...
#cross_val_score(xgb_model,X_train_sc,y_train,cv=10).mean()

/Users/jasonchia/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


0.8162284972411555

<ul>
    
- ```cross_val_score``` was 0.82 already!

In [12]:
#this is for initial decision trees tuned
params = {
    'learning_rate' : [0.01, 0.1, 0.3, 0.5, 0.9],
    'n_estimators' : [20,50,100,200,500,1000],
    'max_depth':[3,5,7,10],
    'gamma':[0,1,2,3],
    'max_delta_step':[1,2,3],
    'scale_pos_weight':[0.5, 1, 2, 3]
    } 

In [14]:
#took too long to run...to be run as part of future plans for this project
#xgb_gridsearch = GridSearchCV(xgb_model, params, cv = 10, verbose = 1, n_jobs = -1)
#xgb_gridsearch.fit(X_train_sc, y_train)

In [ ]:
#print('Gridsearch best score: ', xgb_gridsearch.best_score_)
#print('Gridsearch best estimator: ', xgb_gridsearch.best_estimator_)
#print('Gridsearch best score on test set: ', xgb_gridsearch.best_estimator_.score(X_test_sc,y_test))

In [ ]:
#filename = 'xgb_model.sav'

In [ ]:
#joblib.dump(xgb_gridsearch,filename)

In [ ]:
#loaded_model = joblib.load('xgb_model.sav')

In [ ]:
#loaded_model.best_estimator_.score(X_test_sc,y_test)

In [ ]:
#loaded_model.best_params_

## Formulating recommendations for userid 2043 based on XGB model
---

In [ ]:
#X_test.shape

In [ ]:
#stacking X_test as first step in regenerating the shops column for predictions
#trial = X_test.stack()

In [ ]:
#creating loop to re-generate original X_test order of shops
#index_lst = []
#outlets_lst = []
#for n in range(len(trial.index)):
#    if trial.index[n][1].startswith('shops_') and trial[n]!=0:
#        index_lst.append(str(trial.index[n][0]))
#        outlets_lst.append(trial.index[n][1])
#index_lst = [int(x) for x in index_lst]
#reconstructed_X_test = pd.DataFrame({'shops':outlets_lst}, index=index_lst)

In [ ]:
#rating_predictions = loaded_model.best_estimator_.predict(X_test_sc)

In [ ]:
#reconstructed_X_test['predicted_ratings']=rating_predictions

In [ ]:
#reconstructed_X_test.head()

In [ ]:
#reconstructed_X_test['predicted_ratings'].value_counts()

In [ ]:
#y_test.shape

In [ ]:
#reconstructed_X_test['actual_ratings']=y_test

In [ ]:
#top 5 recommendations for userid 2043 based on content-based filtering (using decision tree classifier) predictions
#reconstructed_X_test.sort_values('predicted_ratings', ascending=False).head(5)

## Defining functions for evaluation of model
---

In [ ]:
# #defining function for obtaining tn, fp, fn, tp for each rating class for feeding into micro-avg precision and recall functions defined below
# def cm_spec(y_true,y_pred,rating,state):
#     if state=='tn':
#         return multilabel_confusion_matrix(y_true,y_pred)[rating-1][0][0]
#     elif state=='fp':
#         return multilabel_confusion_matrix(y_true,y_pred)[rating-1][0][1]
#     elif state=='fn':
#         return multilabel_confusion_matrix(y_true,y_pred)[rating-1][1][0]
#     else:
#         return multilabel_confusion_matrix(y_true,y_pred)[rating-1][1][1]

In [ ]:
# #defining function for obtaining micro-avg precision
# def micro_avg_precision(y_true,y_pred):
#     return ((cm_spec(y_true,y_pred,1,'tp')+
#                                                  cm_spec(y_true,y_pred,2,'tp')+
#                                                  cm_spec(y_true,y_pred,3,'tp')+
#                                                  cm_spec(y_true,y_pred,4,'tp')+
#                                                  cm_spec(y_true,y_pred,5,'tp'))/(
#                                                 cm_spec(y_true,y_pred,1,'tp')+
#                                                  cm_spec(y_true,y_pred,2,'tp')+
#                                                  cm_spec(y_true,y_pred,3,'tp')+
#                                                  cm_spec(y_true,y_pred,4,'tp')+
#                                                  cm_spec(y_true,y_pred,5,'tp')+
#                                                 cm_spec(y_true,y_pred,1,'fp')+
#                                                  cm_spec(y_true,y_pred,2,'fp')+
#                                                  cm_spec(y_true,y_pred,3,'fp')+
#                                                  cm_spec(y_true,y_pred,4,'fp')+
#                                                  cm_spec(y_true,y_pred,5,'fp')))

In [ ]:
# #defining function for obtaining micro-avg recall
# def micro_avg_recall(y_true,y_pred):
#     return ((cm_spec(y_true,y_pred,1,'tp')+
#                                                  cm_spec(y_true,y_pred,2,'tp')+
#                                                  cm_spec(y_true,y_pred,3,'tp')+
#                                                  cm_spec(y_true,y_pred,4,'tp')+
#                                                  cm_spec(y_true,y_pred,5,'tp'))/(
#                                                 cm_spec(y_true,y_pred,1,'tp')+
#                                                  cm_spec(y_true,y_pred,2,'tp')+
#                                                  cm_spec(y_true,y_pred,3,'tp')+
#                                                  cm_spec(y_true,y_pred,4,'tp')+
#                                                  cm_spec(y_true,y_pred,5,'tp')+
#                                                 cm_spec(y_true,y_pred,1,'fn')+
#                                                  cm_spec(y_true,y_pred,2,'fn')+
#                                                  cm_spec(y_true,y_pred,3,'fn')+
#                                                  cm_spec(y_true,y_pred,4,'fn')+
#                                                  cm_spec(y_true,y_pred,5,'fn')))

In [ ]:
# #defining function for obtaining micro_avg_f1
# def micro_avg_f1(y_true,y_pred):
#     return 2 * ((micro_avg_precision(y_true,y_pred) * micro_avg_recall(y_true,y_pred))/(micro_avg_precision(y_true,y_pred) + micro_avg_recall(y_true,y_pred)))

In [ ]:
# def confusion_breakdown(y_true,y_pred,rating):
#     print("True negatives for rating {}: {}".format(
#         rating,multilabel_confusion_matrix(y_true,y_pred)[rating-1][0][0]))
#     print("False positives for rating {}: {}".format(
#         rating,multilabel_confusion_matrix(y_true,y_pred)[rating-1][0][1]))
#     print("False negatives for rating {}: {}".format(
#         rating,multilabel_confusion_matrix(y_true,y_pred)[rating-1][1][0]))
#     print("True positives for rating {}: {}".format(
#         rating,multilabel_confusion_matrix(y_true,y_pred)[rating-1][1][1]))
#     return "******************************************"

In [ ]:
#print(confusion_breakdown(y_test,rating_predictions,1))
#print(confusion_breakdown(y_test,rating_predictions,2))
#print(confusion_breakdown(y_test,rating_predictions,3))
#print(confusion_breakdown(y_test,rating_predictions,4))
#print(confusion_breakdown(y_test,rating_predictions,5))

In [ ]:
#y_test.value_counts()

In [ ]:
#print("Gridsearched XGB yielded micro_avg_precision of ", micro_avg_precision(y_test,rating_predictions))

In [ ]:
#print("Gridsearched XGB yielded micro_avg_recall of ", micro_avg_recall(y_test,rating_predictions))

In [ ]:
#print("Gridsearched XGB yielded micro_avg_f1 of ", micro_avg_f1(y_test,rating_predictions))

In [ ]:
#print(classification_report(y_test,rating_predictions))

In [ ]:
# Generate the probabilities of predictions for above model
#pred_proba_1 = [i[0] for i in loaded_model.best_estimator_.predict_proba(X_test_sc)]
#pred_proba_2 = [i[1] for i in loaded_model.best_estimator_.predict_proba(X_test_sc)]
#pred_proba_3 = [i[2] for i in loaded_model.best_estimator_.predict_proba(X_test_sc)]
#pred_proba_4 = [i[3] for i in loaded_model.best_estimator_.predict_proba(X_test_sc)]
#pred_proba_5 = [i[4] for i in loaded_model.best_estimator_.predict_proba(X_test_sc)]
#pred_proba_all = pred_proba_1 + pred_proba_2 + pred_proba_3 + pred_proba_4 + pred_proba_5

In [ ]:
# Create dataframes with the true values and probabilities for each rating value
#pred_df_1 = pd.DataFrame({'true_values': y_test,
 #                       'pred_probs':pred_proba_1})
#pred_df_2 = pd.DataFrame({'true_values': y_test,
#                        'pred_probs':pred_proba_2})
#pred_df_3 = pd.DataFrame({'true_values': y_test,
#                        'pred_probs':pred_proba_3})
#pred_df_4 = pd.DataFrame({'true_values': y_test,
#                        'pred_probs':pred_proba_4})
#pred_df_5 = pd.DataFrame({'true_values': y_test,
#                        'pred_probs':pred_proba_5})

In [ ]:
#reformating the pred_df for each of the 5 rating classes
#pred_df_1['true_values'] = pred_df_1['true_values'].apply(lambda val:0 if val!=1 else 1)
#pred_df_2['true_values'] = pred_df_2['true_values'].apply(lambda val:0 if val!=2 else 1)
#pred_df_3['true_values'] = pred_df_3['true_values'].apply(lambda val:0 if val!=3 else 1)
#pred_df_4['true_values'] = pred_df_4['true_values'].apply(lambda val:0 if val!=4 else 1)
#pred_df_5['true_values'] = pred_df_5['true_values'].apply(lambda val:0 if val!=5 else 1)

In [ ]:
#creating a pred_df for all classes as a whole for computing the micro-avg ROC AUC later on...
#pred_df_all = pd.DataFrame({'true_values': pred_df_1['true_values'].tolist()+pred_df_2['true_values'].tolist()+
 #                           pred_df_3['true_values'].tolist()+pred_df_4['true_values'].tolist()+
 #                           pred_df_5['true_values'].tolist(),
 #                       'pred_probs':pred_proba_all})

In [ ]:
# Calculate roc score for the respective rating values
#roc_score_1 = roc_auc_score(pred_df_1['true_values'], pred_df_1['pred_probs'], multi_class='ovo')
#print("The ROC AUC for rating 1 is \n{}".format(roc_score_1))
#roc_score_2 = roc_auc_score(pred_df_2['true_values'], pred_df_2['pred_probs'], multi_class='ovo')
#print("The ROC AUC for rating 2 is \n{}".format(roc_score_2))
#roc_score_3 = roc_auc_score(pred_df_3['true_values'], pred_df_3['pred_probs'], multi_class='ovo')
#print("The ROC AUC for rating 3 is \n{}".format(roc_score_3))
#roc_score_4 = roc_auc_score(pred_df_4['true_values'], pred_df_4['pred_probs'], multi_class='ovo')
#print("The ROC AUC for rating 4 is \n{}".format(roc_score_4))
#roc_score_5 = roc_auc_score(pred_df_5['true_values'], pred_df_5['pred_probs'], multi_class='ovo')
#print("The ROC AUC for rating 5 is \n{}".format(roc_score_5))

In [ ]:
#roc_score_all = roc_auc_score(pred_df_all['true_values'], pred_df_all['pred_probs'], multi_class='ovo')
#print("The ROC AUC for all ratings in general is \n{}".format(roc_score_all))

In [ ]:
#Define function to calculate sensitivity. (True positive rate.)
# def TPR(df, true_col, pred_prob_col, threshold):
#     true_positive = df[(df[true_col] == 1) & (df[pred_prob_col] >= threshold)].shape[0]
#     false_negative = df[(df[true_col] == 1) & (df[pred_prob_col] < threshold)].shape[0]
#     return true_positive / (true_positive + false_negative)

# # Define function to calculate 1 - specificity. (False positive rate.)
# def FPR(df, true_col, pred_prob_col, threshold):
#     true_negative = df[(df[true_col] == 0) & (df[pred_prob_col] <= threshold)].shape[0]
#     false_positive = df[(df[true_col] == 0) & (df[pred_prob_col] > threshold)].shape[0]
#     return 1 - (true_negative / (true_negative + false_positive))

# # Define function to calculate micro-averaged sensitivity. (Micro-averaged True positive rate.)
# def micro_avg_TPR(df1,df2,df3,df4,df5, true_col, pred_prob_col, threshold):
#     true_positive1 = df1[(df1[true_col] == 1) & (df1[pred_prob_col] >= threshold)].shape[0]
#     true_positive2 = df2[(df2[true_col] == 1) & (df2[pred_prob_col] >= threshold)].shape[0]
#     true_positive3 = df3[(df3[true_col] == 1) & (df3[pred_prob_col] >= threshold)].shape[0]
#     true_positive4 = df4[(df4[true_col] == 1) & (df4[pred_prob_col] >= threshold)].shape[0]
#     true_positive5 = df5[(df5[true_col] == 1) & (df5[pred_prob_col] >= threshold)].shape[0]
#     false_negative1 = df1[(df1[true_col] == 1) & (df1[pred_prob_col] < threshold)].shape[0]
#     false_negative2 = df2[(df2[true_col] == 1) & (df2[pred_prob_col] < threshold)].shape[0]
#     false_negative3 = df3[(df3[true_col] == 1) & (df3[pred_prob_col] < threshold)].shape[0]
#     false_negative4 = df4[(df4[true_col] == 1) & (df4[pred_prob_col] < threshold)].shape[0]
#     false_negative5 = df5[(df5[true_col] == 1) & (df5[pred_prob_col] < threshold)].shape[0]
#     return (true_positive1+true_positive2+true_positive3+true_positive4+true_positive5) / (true_positive1+true_positive2+true_positive3+true_positive4+true_positive5 + 
#  false_negative1+false_negative2+false_negative3+false_negative4+false_negative5)

# # Define function to calculate micro-averaged false positive rate. (Micro-averaged (1-specificity))
# def micro_avg_FPR(df1,df2,df3,df4,df5, true_col, pred_prob_col, threshold):
#     true_negative1 = df1[(df1[true_col] == 0) & (df1[pred_prob_col] <= threshold)].shape[0]
#     true_negative2 = df2[(df2[true_col] == 0) & (df2[pred_prob_col] <= threshold)].shape[0]
#     true_negative3 = df3[(df3[true_col] == 0) & (df3[pred_prob_col] <= threshold)].shape[0]
#     true_negative4 = df4[(df4[true_col] == 0) & (df4[pred_prob_col] <= threshold)].shape[0]
#     true_negative5 = df5[(df5[true_col] == 0) & (df5[pred_prob_col] <= threshold)].shape[0]
#     false_positive1 = df1[(df1[true_col] == 0) & (df1[pred_prob_col] > threshold)].shape[0]
#     false_positive2 = df2[(df2[true_col] == 0) & (df2[pred_prob_col] > threshold)].shape[0]
#     false_positive3 = df3[(df3[true_col] == 0) & (df3[pred_prob_col] > threshold)].shape[0]
#     false_positive4 = df4[(df4[true_col] == 0) & (df4[pred_prob_col] > threshold)].shape[0]
#     false_positive5 = df5[(df5[true_col] == 0) & (df5[pred_prob_col] > threshold)].shape[0]
#     return 1 - ((true_negative1+true_negative2+true_negative3+true_negative4+true_negative5) / 
# (true_negative1+true_negative2+true_negative3+true_negative4+true_negative5 + 
#  false_positive1+false_positive2+false_positive3+false_positive4+false_positive5))



# # Function to plot the ROC curve
# def roc_plot(fpr_values_1,tpr_values_1,
#              fpr_values_2,tpr_values_2,
#              fpr_values_3,tpr_values_3,
#              fpr_values_4,tpr_values_4,
#              fpr_values_5,tpr_values_5,
#              fpr_values_all,tpr_values_all):
#     # Create figure.
#     plt.figure(figsize = (10,7))

#     # Create threshold values. (Dashed red line in image.)
#     thresholds = np.linspace(0, 1, 200)    

#     # Plot ROC curve for rating 1.
#     plt.plot(fpr_values_1, # False Positive Rate on X-axis
#              tpr_values_1, # True Positive Rate on Y-axis
#              label='Rating 1 ROC Curve with AUC = {}'.format(round(roc_score_1,3)))
    
#     # Plot ROC curve for rating 2.
#     plt.plot(fpr_values_2, # False Positive Rate on X-axis
#              tpr_values_2, # True Positive Rate on Y-axis
#              label='Rating 2 ROC Curve with AUC = {}'.format(round(roc_score_2,3)))
    
#     # Plot ROC curve for rating 3.
#     plt.plot(fpr_values_3, # False Positive Rate on X-axis
#              tpr_values_3, # True Positive Rate on Y-axis
#              label='Rating 3 ROC Curve with AUC = {}'.format(round(roc_score_3,3)))
    
#     # Plot ROC curve for rating 4.
#     plt.plot(fpr_values_4, # False Positive Rate on X-axis
#              tpr_values_4, # True Positive Rate on Y-axis
#              label='Rating 4 ROC Curve with AUC = {}'.format(round(roc_score_4,3)))
    
#     # Plot ROC curve for rating 5.
#     plt.plot(fpr_values_5, # False Positive Rate on X-axis
#              tpr_values_5, # True Positive Rate on Y-axis
#              label='Rating 5 ROC Curve with AUC = {}'.format(round(roc_score_5,3)))
    
#     # Plot ROC curve for all ratings in general (micro-averaged).
#     plt.plot(fpr_values_all, # False Positive Rate on X-axis
#              tpr_values_all, # True Positive Rate on Y-axis
#              label='Micro-Averaged ROC Curve with AUC = {}'.format(round(roc_score_all,3)))
    
    
#     # Plot baseline. (Perfect overlap between the two populations.)
#     plt.plot(np.linspace(0, 1, 200),
#              np.linspace(0, 1, 200),
#              label='baseline',
#              linestyle='--')

#     # Label axes.
#     #plt.title(f'ROC Curve with AUC = {round(roc_auc_score(pred_df['true_values'],pred_df['pred_probs']),3)}', fontsize=22)
#     plt.title('Multi-class ROC AUC', fontsize=15)
#     plt.ylabel('Sensitivity', fontsize=12)
#     plt.xlabel('1 - Specificity', fontsize=12)

#     # Create legend.
#     plt.legend(fontsize=10);

In [ ]:
# # Calculate sensitivity & 1-specificity for each threshold between 0 and 1.

# # Create threshold values. (Dashed red line in image.)
# thresholds = np.linspace(0, 1, 200)  

# # Call the function to create true_positive_rate values
# tpr_values_1 = [TPR(pred_df_1, 'true_values', 'pred_probs', prob) for prob in thresholds]
# tpr_values_2 = [TPR(pred_df_2, 'true_values', 'pred_probs', prob) for prob in thresholds]
# tpr_values_3 = [TPR(pred_df_3, 'true_values', 'pred_probs', prob) for prob in thresholds]
# tpr_values_4 = [TPR(pred_df_4, 'true_values', 'pred_probs', prob) for prob in thresholds]
# tpr_values_5 = [TPR(pred_df_5, 'true_values', 'pred_probs', prob) for prob in thresholds]
# tpr_values_all = [micro_avg_TPR(pred_df_1,pred_df_2,pred_df_3,pred_df_4,pred_df_5, 'true_values', 'pred_probs', prob) for prob in thresholds]
# # Call the function to create false_positive_rate values
# fpr_values_1 = [FPR(pred_df_1, 'true_values', 'pred_probs', prob) for prob in thresholds]
# fpr_values_2 = [FPR(pred_df_2, 'true_values', 'pred_probs', prob) for prob in thresholds]
# fpr_values_3 = [FPR(pred_df_3, 'true_values', 'pred_probs', prob) for prob in thresholds]
# fpr_values_4 = [FPR(pred_df_4, 'true_values', 'pred_probs', prob) for prob in thresholds]
# fpr_values_5 = [FPR(pred_df_5, 'true_values', 'pred_probs', prob) for prob in thresholds]
# fpr_values_all = [micro_avg_FPR(pred_df_1,pred_df_2,pred_df_3,pred_df_4,pred_df_5, 'true_values', 'pred_probs', prob) for prob in thresholds]

# # Plot the graph
# roc_plot(fpr_values_1,tpr_values_1,
#          fpr_values_2,tpr_values_2,
#          fpr_values_3,tpr_values_3,
#          fpr_values_4,tpr_values_4,
#          fpr_values_5,tpr_values_5,
#          fpr_values_all,tpr_values_all)

<ul>
    
- Tuned XGBClassifier______